![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/blogposts/6.Comparison_of_Medical_and_Large_LLMs_in_RAG.ipynb)

# Comparison of Medical and Large LLMs in RAG

# Colab Setup

In [ ]:
import json
import os
from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.1 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [ ]:
import functools
import numpy as np
import pandas as pd
from scipy import spatial

import pyspark.sql.types as T
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

## LangChain Embeddings

In [ ]:
!pip install -q --upgrade tiktoken faiss-cpu

In [ ]:
!pip install -Uq langchain-community langchain-openai

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize embeddings using a Hugging Face model
model_name = 'hkunlp/instructor-base'  # Example of a Hugging Face model similar to 'instructor_base'

# Create an embeddings object
embeddings = HuggingFaceEmbeddings(model_name=model_name)

## FAISS

**Creating Vector DB with PubMed dataset :**

First, we download a dataset about diabetes research, loading it into a Spark DataFrame. We split the documents into manageable chunks and convert them into numerical vectors using embeddings. Then, we use FAISS to create a database for similarity search. Finally, the FAISS retriever fetches the top relevant document chunks, ensuring information retrieval.

```python
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/healthcare-nlp/data/pubmed_diabetes_1000_meta.csv

df = spark.createDataFrame(pd.read_csv("./pubmed_diabetes_1000_meta.csv", usecols=range(1,6)))
df.orderBy("pubmed_id").write.mode("overwrite").parquet('pubmed_diabetes_1000_meta.parquet')

from langchain.document_loaders import PySparkDataFrameLoader
loader = PySparkDataFrameLoader(spark, df, page_content_column="abstract")
documents = loader.load()

from johnsnowlabs.llm import embedding_retrieval
embeddings =  embedding_retrieval.JohnSnowLabsLangChainEmbedder('en.embed_sentence.instructor_base')

from johnsnowlabs.llm import embedding_retrieval
jsl_splitter = embedding_retrieval.JohnSnowLabsLangChainCharSplitter(chunk_size=500,
                                                                     chunk_overlap=50)
texts = jsl_splitter.split_documents(documents)

!pip install -q faiss-cpu
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embeddings)
faiss_retriever = db.as_retriever(search_kwargs={"k": 5})
!zip -r faiss_retriever.zip faiss_retriever

```

**Load a Preset Database:**

You can also utilize a pre-configured vector database.



In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
!wget -q https://github.com/JohnSnowLabs/spark-nlp-workshop/raw/master/healthcare-nlp/data/faiss_retriever_db_diabetes.vs.zip -O faiss_retriever_db_diabetes.vs.zip

In [ ]:
import zipfile

# Path to your zip file
zip_path = "faiss_retriever_db_diabetes.vs.zip"
# Destination folder
extract_path = "."

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
# Load your FAISS index
db = FAISS.load_local("faiss_retriever_db_diabetes.vs", embeddings, allow_dangerous_deserialization=True)

## Utilities

In [ ]:
SYSTEM_PROMPT = """Use the following pieces of context to answer the question. Do not explain detail.
If you don't know the answer, just say that you don't know, don't try to make up an answer."""

Here is the list of questions pertaining to the diabetes dataset for generating prompt.

In [ ]:
q_list = [
    "What are the recommended measures to reduce the risk of GDM in pregnant women?",
    "What are the main symptoms of diabetes?",
    "What dietary changes can help manage diabetes?",
    "What are the clinical symptoms associated with Wolfram syndrome?",
    "What are some low-cost ways to screen individuals for prediabetes?",
    "What are the most common malignancies associated with pituitary spread?",
    "What is the recommended total daily dose of hydrocortisone for adults with adrenal insufficiency?"
]

`process_question` function retrieves relevant top documents from FAISS database based on queries. The function constructs a prompt that combines the context from the retrieved documents with the question.

In [ ]:
def process_question(question, db, top_k=5):
    # Create a retriever from the FAISS database (db)
    faiss_retriever = db.as_retriever(search_kwargs={"k": top_k})

    # Retrieve relevant documents for the current question
    results = faiss_retriever.get_relevant_documents(question)

    # Initialize merged_content for this question
    merged_content = ""

    # Loop through the results to accumulate page contents
    for result in results:
        # Extract the page content and append it to merged_content
        page_content = result.page_content
        merged_content += f"{page_content}\n"

    # Add the question at the end
    RAG_PROMPT = f"### Context: \n{merged_content}\n### Questions:\n{question}\n"

    return RAG_PROMPT

# Medical LLMs

| Model Name              | Description |
|-------------------------|-------------|
|[jsl_meds_ner_q16_v2](https://nlp.johnsnowlabs.com/2024/08/21/jsl_meds_ner_q16_v2_en.html)    | Extract and link medical named entities |
|[jsl_meds_ner_q8_v2](https://nlp.johnsnowlabs.com/2024/08/21/jsl_meds_ner_q8_v2_en.html)      | Extract and link medical named entities |
|[jsl_meds_ner_q4_v2](https://nlp.johnsnowlabs.com/2024/08/21/jsl_meds_ner_q4_v2_en.html)      | Extract and link medical named entities |
|[jsl_medsner_zs_q16_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medsner_zs_q16_v1_en.html)| Extract and link medical named entities |
|[jsl_medsner_zs_q8_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medsner_zs_q8_v1_en.html)  | Extract and link medical named entities |
|[jsl_medsner_zs_q4_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medsner_zs_q4_v1_en.html)  | Extract and link medical named entities |
|[jsl_meds_q16_v3](https://nlp.johnsnowlabs.com/2024/09/19/jsl_meds_q16_v3_en.html)            | Summarization, Q&A, RAG |
|[jsl_meds_q8_v3](https://nlp.johnsnowlabs.com/2024/09/19/jsl_meds_q8_v3_en.html)              | Summarization, Q&A, RAG |
|[jsl_meds_q4_v3](https://nlp.johnsnowlabs.com/2024/09/19/jsl_meds_q4_v3_en.html)              | Summarization, Q&A, RAG |
|[jsl_meds_q16_v2](https://nlp.johnsnowlabs.com/2024/09/19/jsl_meds_q16_v2_en.html)            | Summarization, Q&A, RAG |
|[jsl_meds_q8_v2](https://nlp.johnsnowlabs.com/2024/09/19/jsl_meds_q8_v2_en.html)              | Summarization, Q&A, RAG |
|[jsl_meds_q4_v2](https://nlp.johnsnowlabs.com/2024/09/19/jsl_meds_q4_v2_en.html)              | Summarization, Q&A, RAG |
|[jsl_meds_q16_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_meds_q16_v1_en.html)            | Summarization, Q&A, RAG |
|[jsl_meds_q8_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_meds_q8_v1_en.html)              | Summarization, Q&A, RAG|
|[jsl_meds_q4_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_meds_q4_v1_en.html)              | Summarization, Q&A, RAG |
|[jsl_medm_q16_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medm_q16_v1_en.html)            | Summarization, Q&A, RAG, and Chat |
|[jsl_medm_q8_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medm_q8_v1_en.html)              | Summarization, Q&A, RAG, and Chat |
|[jsl_medm_q4_v1](https://nlp.johnsnowlabs.com/2024/07/12/jsl_medm_q4_v1_en.html)              | Summarization, Q&A, RAG, and Chat |
|[jsl_meds_rag_q4_v1](https://nlp.johnsnowlabs.com/2024/08/21/jsl_meds_rag_q4_v1_en.html)     | LLM component of Retrieval Augmented Generation (RAG) |
|[jsl_meds_rag_q8_v1](https://nlp.johnsnowlabs.com/2024/08/21/jsl_meds_rag_q8_v1_en.html)      | LLM component of Retrieval Augmented Generation (RAG) |
|[jsl_meds_rag_q16_v1](https://nlp.johnsnowlabs.com/2024/08/21/jsl_meds_rag_q16_v1_en.html)     | LLM component of Retrieval Augmented Generation (RAG) |

**We recommend using 8b quantized versions of the models as the qualitative performance difference between q16 and q8 versions is very negligible.**

# LLMLoader

`LLMLoader` is designed to interact with a LLMs that are converted into gguf format. This module allows using John Snow Labs' licensed LLMs at various sizes that are finetuned on medical context for certain tasks. It provides various methods for setting parameters, loading models, generating text, and retrieving metadata. The `LLMLoader` includes methods for setting various parameters such as input prefix, suffix, cache prompt, number of tokens to predict, sampling techniques, temperature, penalties, and more. Overall, the `LLMLoader`  provides a flexible and extensible framework for interacting with language models in a Python and Scala environment using PySpark and Java.

The `LLMLoader`, now based on the llama cpp dependency, allows loading models (either `AutoGGUFModel` or `MedicalLLM`) via pretrained and load methods. It can also load any other gguf model outside of the Models Hub. It automatically detects whether a model is licensed and can also load GGUF files directly using the `loadGGUF` method. This streamlines model loading and usage without requiring the inclusion of the llama cpp dependency.

# Load LLMs

Due to the high memory requirements of LLM models, it's advisable to use each model individually and restart your system before trying other models.

## JSL_MedS_Rag

This LLM model is trained to perform Q&A, Summarization, RAG, and Chat.

In [ ]:
from sparknlp_jsl.llm import LLMLoader

jsl_meds_rag_llm = LLMLoader(spark).pretrained("jsl_meds_rag_q8_v1", "en", "clinical/models")

## JSL_MedS


This LLM model is trained to perform Summarization and Q&A based on a given context.

In [ ]:
from sparknlp_jsl.llm import LLMLoader

jsl_meds_llm = LLMLoader(spark).pretrained("jsl_meds_q8_v3", "en", "clinical/models")

## JSL_MedM


This LLM model is trained to perform Q&A, Summarization, RAG, and Chat.

In [ ]:
from sparknlp_jsl.llm import LLMLoader

jsl_medm_llm = LLMLoader(spark).pretrained("jsl_medm_q8_v2", "en", "clinical/models")

## JSL-MedM-Rag

This LLM model is trained to perform Q&A, Summarization, RAG, and Chat


In [ ]:
from sparknlp_jsl.llm import LLMLoader

jsl_medm_rag_llm = LLMLoader(spark).pretrained("jsl_medm_rag_q8_v1", "en", "clinical/models")

## GPT-4

`gpt-4o` is a state-of-the-art variant of OpenAI's GPT-4 model.

In [ ]:
from openai import OpenAI

In [ ]:
from getpass import getpass
OPENAI_API_KEY = getpass('Please enter your open_api_key:')

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def generate_response(RAG_PROMPT, gpt_model_name = "gpt-4o", temperature=0.1):
  response = client.chat.completions.create(
                  model=gpt_model_name,
                  temperature=temperature,
                  messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": RAG_PROMPT},
                  ]
                )
  return response

## LongCite

`THUDM/LongCite-llama3.1-8b` is a large language model (LLM) developed by Tsinghua University, specifically designed to address long-context question-answering (QA).

In [ ]:
!pip install -q -U transformers optimum
!pip install tiktoken

In [ ]:
import tiktoken
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained('THUDM/LongCite-llama3.1-8b', trust_remote_code=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained('THUDM/LongCite-llama3.1-8b', torch_dtype=torch.bfloat16, trust_remote_code=True, device_map='auto')


# Questions and Responses

After loading each LLM model individually, you can get responses for each question. Make sure to retrieve the RAG prompt each time.

Below are responses generated by LLMs to questions related to a diabetes dataset, along with the RAG outputs.

## Q1

In [ ]:
question = "What are the recommended measures to reduce the risk of GDM in pregnant women?"
RAG_PROMPT = process_question(question, db, top_k=5)

In [ ]:
print(RAG_PROMPT)

### Context: 
of metabolic disease. One of the strongest risk factors for GDM is the occurrence of GDM in a prior pregnancy. Preliminary data from epidemiologic and bariatric surgery studies suggest that reducing body weight before pregnancy can prevent the development of GDM, but no adequately powered trial has tested the effects of a maternal lifestyle intervention before pregnancy to reduce body weight and prevent GDM recurrence.
outcomes and prevent GDM and associated health risks.
Implementation of dietary and lifestyle interventions prior to and early in pregnancy in high risk women has been shown to reduce the risk of gestational diabetes mellitus (GDM) development later in pregnancy. Although numerous risk factors for GDM have been identified, the ability to accurately identify women before or early in pregnancy who could benefit most from these interventions remains limited. As nulliparous women are an under-screened population with risk profiles that differ from
prenatal visi

### JSL_MedS_Rag

In [ ]:
response = jsl_meds_rag_llm.generate(RAG_PROMPT)
response

'To reduce the risk of gestational diabetes mellitus (GDM) in pregnant women, the following measures are recommended:\n\n1. Prepregnancy weight management: Women with a high prepregnancy body mass index (BMI) and excessive weight gain before the 24th week of pregnancy should aim to reduce their weight and maintain a healthy BMI. This can be achieved through a balanced diet and regular physical activity.\n\n2. Prepregnancy lifestyle interventions: Women at high risk of GDM should consider implementing dietary and lifestyle interventions before and early in pregnancy. These interventions may include a healthy diet, regular exercise, and stress management techniques.\n\n3. Screening for GDM: Women with risk factors such as a history of GDM, obesity, or a large baby should be screened for GDM early in pregnancy. This can help identify women who may benefit from lifestyle interventions and allow for early management of the condition.\n\n4. Monitoring and managing blood glucose levels: Women

### JSL_MedS

In [ ]:
response = jsl_meds_llm.generate(RAG_PROMPT)
response

The recommended measures to reduce the risk of gestational diabetes mellitus (GDM) in pregnant women include:
1. Implementing dietary and lifestyle interventions prior to and early in pregnancy, especially in high-risk women.
2. Reducing body weight before pregnancy, as preliminary data suggest that this can prevent the development of GDM.
3. Identifying risk factors for GDM, such as a history of GDM in a prior pregnancy, low birth weight (<3000 g), and prepregnancy BMI and weight gain before the 24th week.
4. Ensuring birth weight above 3000 g to help reduce the risk for future GDM among female offspring.
5. Screening and monitoring high-risk women, including those with a history of GDM, low birth weight, and prepregnancy BMI and weight gain before the 24th week.
6. Providing appropriate medical care and interventions to manage and prevent GDM in pregnant women.


### JSL_MedM

In [ ]:
response = jsl_medm_llm.generate(RAG_PROMPT)
response

'The recommended measures to reduce the risk of gestational diabetes mellitus (GDM) in pregnant women include:\n\n1. Reducing body weight before pregnancy: Preliminary data from epidemiologic and bariatric surgery studies suggest that reducing body weight before pregnancy can prevent the development of GDM.\n2. Implementation of dietary and lifestyle interventions: Implementation of dietary and lifestyle interventions prior to and early in pregnancy in high-risk women has been shown to reduce the risk of GDM development later in pregnancy.\n3. Ensuring birth weight above 3000 g: Ensuring birth weight above 3000 g may help reduce the risk for future GDM among female offspring.\n4. Modifying prepregnancy BMI and weight gain before 24th week: Prepregnancy BMI and weight gain before 24th week are important modifiable risk factors for GDM.\n\nThese measures can help reduce the risk of GDM and associated health risks in pregnant women.'

### JSL-MedM_Rag

In [ ]:
response = jsl_medm_rag_llm.generate(RAG_PROMPT)
response

Based on the context, the recommended measures to reduce the risk of gestational diabetes mellitus (GDM) in pregnant women include:

- Implementing dietary and lifestyle interventions before and early in pregnancy in high risk women. This includes reducing body weight before pregnancy to prevent GDM recurrence.

- Identifying women early in pregnancy who could benefit most from these interventions. Nulliparous women are an under-screened population with risk profiles that differ from multiparous women.

- Ensuring birth weight above 3000 g to reduce risk for future GDM among female offspring.

- Screening for GDM risk factors like prepregnancy BMI and weight gain before 24 weeks.

- Providing dietary and lifestyle interventions to reduce risk of GDM.

So in summary, early identification, weight reduction, dietary and lifestyle changes are key measures recommended to reduce risk of GDM in pregnant women. Ensuring birth weight above 3000 g may also reduce risk for future GDM.


### GPT-4

In [ ]:
# Generate the response using the model
response = generate_response(RAG_PROMPT)

In [ ]:
response_content = response.choices[0].message.content

print(response_content)

The recommended measures to reduce the risk of GDM in pregnant women include implementing dietary and lifestyle interventions prior to and early in pregnancy, reducing body weight before pregnancy, and managing prepregnancy BMI and weight gain before the 24th week of pregnancy.


### LongCite

In [ ]:
context = RAG_PROMPT
query = "What are the recommended measures to reduce the risk of GDM in pregnant women?"
result = model.query_longcite(context, query, tokenizer=tokenizer, max_input_length=128000, max_new_tokens=512)

In [ ]:
answer = result['answer']
cleaned_answer = answer.replace("<statement>", "").replace("</statement>", "")
print(cleaned_answer)

Based on the context, some recommended measures to reduce the risk of GDM in pregnant women include:<cite></cite>

- Implementing dietary and lifestyle interventions prior to and early in pregnancy in high risk women to reduce risk of GDM later in pregnancy. This includes interventions to reduce body weight before pregnancy.<cite>[2-2][4-4][9-9]</cite>

- Identifying and targeting women who are at high risk for GDM before or early in pregnancy. Nulliparous women are an under-screened population with risk profiles that differ from those of multiparous women.<cite>[5-6]</cite>

- Ensuring birth weight above 3000 g may help reduce risk for future GDM among female offspring.<cite>[10-10]</cite>

- Managing prepregnancy BMI and weight gain before 24th week are important modifiable risk factors for GDM.<cite>[9-9]</cite>

- Screening for GDM and DIP in pregnant women to identify and manage these conditions early.<cite></cite>

- Providing education and support to pregnant women to make lifes

## Q2

In [ ]:
question = "What are the main symptoms of diabetes?"
RAG_PROMPT = process_question(question, db, top_k=5)

In [ ]:
print(RAG_PROMPT)

### Context: 
replacement therapy, the symptoms of diabetes insipidus appeared.
Almost all (99%) of the students had knowledge about diabetes mellitus. The majority (83.1%) reported that diabetes mellitus is not completely a genetic/hereditary disease. Only a minority of respondents reported that they should worry about diabetes before 45 years of age. Common symptoms of diabetes reported by the respondents included constant hunger, blurred vision, fatigue, and frequent urination.
Patient: Male, 2-year-old 
 Final Diagnosis: Adipsic diabetes insipidus 
 Symptoms: Dehydration • polyuria 
 Medication: Desmopressin 
 Clinical Procedure: Fluid replacement 
 Specialty: Pediatrics and Neonatology • Endocrine and Metabolic 
 Rare coexistence of disease or pathology
symptoms. She did report feeling hungry. She had no family history of diabetes, normal body mass index of 21.7, and reported taking her thyroid replacement medication every day. The differential diagnosis for her thirst included de

### JSL_MedS_Rag

In [ ]:
response = jsl_meds_rag_llm.generate(RAG_PROMPT)
response

'The main symptoms of diabetes include constant hunger, blurred vision, fatigue, and frequent urination.'

### JSL_MedS

In [ ]:
response = jsl_meds_llm.generate(RAG_PROMPT)
response

"The main symptoms of diabetes include:\n\n1. Constant hunger\n2. Blurred vision\n3. Fatigue\n4. Frequent urination\n5. Excessive thirst (polydipsia)\n6. Unexplained weight loss (in type 1 diabetes)\n7. Increased urination (polyuria)\n8. Presence of ketones in urine (ketoacidosis)\n\nThese symptoms can vary depending on the type of diabetes and the individual's response to high blood sugar levels. It is important for individuals experiencing these symptoms to seek medical evaluation for proper diagnosis and management."

### JSL_MedM

In [ ]:
response = jsl_medm_llm.generate(RAG_PROMPT)
response

'The main symptoms of diabetes include:\n\n1. Constant hunger\n2. Blurred vision\n3. Fatigue\n4. Frequent urination\n\nThese symptoms are commonly reported by patients with diabetes mellitus.'

### JSL-MedM_Rag

In [ ]:
response = jsl_medm_rag_llm.generate(RAG_PROMPT)
response

Based on the patient case and discussion, the main symptoms of diabetes reported by the respondents include:

- Constant hunger 

- Blurred vision

- Fatigue

- Frequent urination

The patient also had dehydration and polyuria. The symptoms of thirst were due to diabetes insipidus. The patient had no family history of diabetes and was taking thyroid replacement medication. Labs revealed a random blood glucose level of 140.

So in summary, the main symptoms of diabetes reported by the respondents are constant hunger, blurred vision, fatigue, frequent urination, dehydration, and polyuria. The patient had no family history of diabetes and was taking thyroid medication. Labs showed a random blood glucose level of 140.


### GPT-4

In [ ]:
# Generate the response using the model
response = generate_response(RAG_PROMPT)

In [ ]:
response_content = response.choices[0].message.content

print(response_content)

The main symptoms of diabetes include constant hunger, blurred vision, fatigue, and frequent urination.


### LongCite

In [ ]:
context = RAG_PROMPT
query = "What are the main symptoms of diabetes?"
result = model.query_longcite(context, query, tokenizer=tokenizer, max_input_length=128000, max_new_tokens=512)

In [ ]:
answer = result['answer']
cleaned_answer = answer.replace("<statement>", "").replace("</statement>", "")
print(cleaned_answer)

Based on the context, the main symptoms of diabetes reported by respondents include:<cite></cite>

- Constant hunger<cite>[3-3]</cite>

- Blurred vision  <cite>[3-3]</cite>

- Fatigue<cite>[3-3]</cite>

- Frequent urination<cite>[3-3]</cite>

The context also mentions that dehydration and polyuria (frequent urination) were the main symptoms in the patient with adipsic diabetes insipidus. So frequent urination is a key symptom. The patient did report feeling hungry, but this was not listed as a main symptom by the respondents.<cite>[3-5]</cite>


## Q3

In [ ]:
question = "What dietary changes can help manage diabetes?"
RAG_PROMPT = process_question(question, db, top_k=5)

In [ ]:
print(RAG_PROMPT)

### Context: 
incidence of type 2 diabetes with higher intake of red meat (for an increment of 100 g/day, 1.17 (1.08 to 1.26)), processed meat (for an increment of 50 g/day, 1.37 (1.22 to 1.54)), bacon (per two slices/day, 2.07 (1.40 to 3.05)), and sugar sweetened beverages (for an increase of one serving/day, 1.26 (1.11 to 1.43)).
of 150 newly diagnosed cases attending a diabetes clinic, and 150 controls, who were community residents and free of diabetes. The case group had a significantly higher consumption of noodles, fish, beans, fermented food and pickles, dried food, topping seasonings, and non-dairy milk products than the control group, whereas they had a lower vegetable intake (more than three servings/day) and fruit intake (more than three servings/day) than the control group. Furthermore, the case group
income, there is a need for a certain affordable dietary pattern to ameliorate the rise in diabetes incidence, as well as improve the glycemic control. We did a review based o

### JSL_MedS_Rag

In [ ]:
response = jsl_meds_rag_llm.generate(RAG_PROMPT)
response

"Dietary changes that can help manage diabetes include adopting a Mediterranean diet, which emphasizes on fruits, vegetables, whole grains, legumes, nuts, and olive oil. It also includes moderate consumption of fish and poultry, and limited intake of red meat and sweets. Additionally, reducing the intake of processed meats, sugar-sweetened beverages, and increasing the consumption of non-dairy milk products, noodles, fish, beans, fermented food, and pickles can be beneficial. It's also important to increase the intake of vegetables and fruits. These dietary changes can improve glycemic control, lipid profile, and metabolic outcomes in people with or at risk of developing type 2 diabetes mellitus."

### JSL_MedS

In [ ]:
response = jsl_meds_llm.generate(RAG_PROMPT)
response

'Dietary changes that can help manage diabetes include:\n\n1. Reducing intake of red meat, processed meat, and sugar-sweetened beverages: Studies have shown that higher consumption of these foods is associated with an increased risk of type 2 diabetes. Limiting their intake can help improve glycemic control and reduce the risk of developing diabetes.\n2. Increasing consumption of vegetables and fruits: A diet rich in vegetables and fruits can help improve glycemic control and reduce the risk of diabetes. It is recommended to consume more than three servings of vegetables and fruits per day.\n3. Incorporating whole grains and legumes: Whole grains and legumes are rich in fiber and can help regulate blood sugar levels. They can be included in the diet in the form of whole wheat bread, brown rice, lentils, and chickpeas.\n4. Limiting intake of refined carbohydrates and sugary foods: Refined carbohydrates and sugary foods can cause spikes in blood sugar levels. It is recommended to limit t

### JSL_MedM

In [ ]:
response = jsl_medm_llm.generate(RAG_PROMPT)
response

'A Mediterranean diet has been shown to be beneficial in managing type 2 diabetes mellitus (T2DM). This diet is characterized by high consumption of fruits, vegetables, whole grains, nuts, seeds, legumes, olive oil, and fish, and low consumption of red meat, processed meat, and sugar-sweetened beverages. The Mediterranean diet has been associated with improved glycemic control, lipid profile, and metabolic outcomes in patients with T2DM. Additionally, lifestyle changes such as improved diet and increased physical activity are important for managing T2DM.'

### JSL-MedM_Rag

In [ ]:
response = jsl_medm_rag_llm.generate(RAG_PROMPT)
response

Based on the provided context, some key dietary changes that can help manage diabetes include:

- Reducing intake of red meat, processed meat, bacon, and sugar sweetened beverages. The context states that these foods are associated with a higher incidence of type 2 diabetes.

- Increasing intake of vegetables and fruits. The context notes that the case group had a lower intake of vegetables and fruits compared to controls.

- Following a Mediterranean diet. The review mentions that a Mediterranean diet is beneficial for glycemic control and metabolic outcomes in people with type 2 diabetes.

- Avoiding high-glycemic index foods. The context does not specifically mention this, but avoiding high glycemic index foods is a general dietary recommendation for managing diabetes.

- Monitoring and controlling portion sizes. The context does not directly mention this, but portion control is a general dietary recommendation for diabetes management.

- Choosing healthier fats. The context does no

### GPT-4

In [ ]:
# Generate the response using the model
response = generate_response(RAG_PROMPT)

In [ ]:
response_content = response.choices[0].message.content

print(response_content)

Dietary changes that can help manage diabetes include adopting a Mediterranean diet, which is associated with improved glycemic control, lipid profile, and metabolic outcomes. Additionally, reducing the intake of red and processed meats, bacon, and sugar-sweetened beverages can be beneficial. Increasing the consumption of vegetables and fruits, while reducing the intake of noodles, fish, beans, fermented foods, pickles, dried foods, topping seasonings, and non-dairy milk products, may also help in managing diabetes.


### LongCite

In [ ]:
context = RAG_PROMPT
query = "What dietary changes can help manage diabetes?"
result = model.query_longcite(context, query, tokenizer=tokenizer, max_input_length=128000, max_new_tokens=512)

In [ ]:
answer = result['answer']
cleaned_answer = answer.replace("<statement>", "").replace("</statement>", "")
print(cleaned_answer)

Based on the context, some key dietary changes that can help manage diabetes include:<cite></cite>

- Reducing intake of red meat, processed meat, bacon, and sugar sweetened beverages. Higher intake of these foods is associated with increased risk of type 2 diabetes.<cite>[0-0]</cite>

- Eating more vegetables and fruits. Higher consumption of noodles, fish, beans, fermented food and pickles, dried food, topping seasonings, and non-dairy milk products was found in the case group compared to controls, while they had a lower vegetable and fruit intake.<cite>[2-2]</cite>

- Following a Mediterranean diet. A review of literature found that the Mediterranean diet has beneficial effects on glycemic control, lipid profile, and metabolic outcome in people with or at risk of developing type 2 diabetes.<cite>[4-4]</cite>

- Limiting portion sizes. For example, limiting intake of red meat to 100g/day, processed meat to 50g/day, and sugar sweetened beverages to one serving/day was associated with 

## Q4

In [ ]:
question = "What are the clinical symptoms associated with Wolfram syndrome?"
RAG_PROMPT = process_question(question, db, top_k=5)

In [ ]:
print(RAG_PROMPT)

### Context: 
Wolframs syndrome being a multisystem degenerative disorder can have myriad other manifestations than the classically described features. Neuropsychiatric manifestations, depression with suicidal risk, central hypogonadism, and secondary adrenal insufficiency are among the less well-described manifestations of this syndrome.
Wolfram syndrome (WFS) is a rare autosomal recessive disease with clinical manifestations of diabetes mellitus (DM), diabetes insipidus (DI), optic nerve atrophy (OA) and sensorineural hearing loss (SNHL). Although SNHL is a key symptom of WFS, there is limited information on its natural history using standardized measures. Such information is important for clinical care and determining its use as an outcome measure in clinical trials.
Wolfram syndrome (WS) is a rare autosomal recessive disorder characterized by diabetes insipidus, diabetes mellitus, optic atrophy and deafness. Mutations in Wolfram syndrome 1 (WFS1) gene may cause dysregulated endopla

### JSL_MedS_Rag

In [ ]:
response = jsl_meds_rag_llm.generate(RAG_PROMPT)
response

"Wolfram syndrome is a rare genetic disorder characterized by several clinical symptoms. These include diabetes mellitus, which is a condition that affects the body's ability to process blood sugar. The syndrome also involves optic atrophy, which is a degenerative eye disease that leads to vision loss. Additionally, individuals with Wolfram syndrome experience sensorineural hearing loss, which is a type of hearing loss that occurs due to damage to the inner ear or the nerve pathways from the ear to the brain. Other symptoms may include diabetes insipidus, neurological disorders, urinary tract anomalies, endocrine dysfunctions, and various systemic manifestations."

### JSL_MedS

In [ ]:
response = jsl_meds_llm.generate(RAG_PROMPT)
response

'Wolfram syndrome (WS) is a rare autosomal recessive disorder characterized by several clinical symptoms. The key symptoms include:\n\n1. Diabetes mellitus (DM): WS patients often develop insulin-dependent diabetes, which can lead to various complications if not properly managed.\n2. Diabetes insipidus (DI): This condition is characterized by excessive urination and thirst due to a deficiency of antidiuretic hormone (ADH) or a resistance to its effects.\n3. Optic nerve atrophy (OA): WS patients may experience vision loss and other visual impairments due to damage to the optic nerve.\n4. Sensorineural hearing loss (SNHL): Hearing loss is a common symptom of WS, which can affect the ability to hear and understand sounds.\n5. Neurological disorders: WS can also cause neurological problems, such as movement disorders, cognitive impairment, and other neurodegenerative processes.\n6. Olfactory dysfunction: Some studies have reported olfactory dysfunction in WS patients, but its clinical sign

### JSL_MedM

In [ ]:
response = jsl_medm_llm.generate(RAG_PROMPT)
response

'Wolfram syndrome is a rare autosomal recessive disease characterized by several clinical symptoms, including:\n\n1. Diabetes mellitus (DM): This is one of the key symptoms of Wolfram syndrome, which is caused by a mutation in the WFS1 gene.\n2. Diabetes insipidus (DI): This is another common symptom of Wolfram syndrome, which is caused by a deficiency of antidiuretic hormone (ADH).\n3. Optic nerve atrophy (OA): This is a degeneration of the optic nerve, which can lead to vision loss and blindness.\n4. Sensorineural hearing loss (SNHL): This is a type of hearing loss that affects the inner ear and can lead to significant hearing impairment.\n5. Neurodegeneration: This is a progressive degeneration of the brain and nervous system, which can lead to a range of cognitive and motor symptoms.\n6. Olfactory dysfunction: This is a loss of smell, which can be a classical clinical marker of neurodegenerative processes.\n7. Taste dysfunction: This is a loss of taste, which is not well-studied in

### JSL-MedM_Rag

In [ ]:
response = jsl_medm_rag_llm.generate(RAG_PROMPT)
response

Based on the context, the key clinical symptoms associated with Wolfram syndrome are:

- Diabetes mellitus - Insulin-dependent diabetes that is a core feature of the syndrome.

- Diabetes insipidus - Excessive thirst and urination due to lack of antidiuretic hormone.

- Optic atrophy - Degeneration of the optic nerve leading to vision loss.

- Sensorineural hearing loss - Progressive hearing loss due to damage to the inner ear and/or auditory nerve.

- Neurodegeneration - Progressive degeneration of the brain and nervous system, which can lead to a variety of neurologic symptoms.

- Other symptoms can include gastrointestinal dysfunction, renal and hepatic dysfunction, urinary tract anomalies, and endocrine dysfunctions.

So in summary, the core clinical symptoms are diabetes, diabetes insipidus, optic atrophy, hearing loss, and neurodegeneration. The syndrome can also involve other systemic manifestations.


### GPT-4

In [ ]:
# Generate the response using the model
response = generate_response(RAG_PROMPT)

In [ ]:
response_content = response.choices[0].message.content

print(response_content)

The clinical symptoms associated with Wolfram syndrome include diabetes mellitus, diabetes insipidus, optic nerve atrophy, sensorineural hearing loss, neuropsychiatric manifestations, depression with suicidal risk, central hypogonadism, secondary adrenal insufficiency, and neurodegeneration. Other possible symptoms include olfactory dysfunction, neurological disorders, urinary tract anomalies, and endocrine dysfunctions.


### LongCite

In [ ]:
context = RAG_PROMPT
query = "What are the clinical symptoms associated with Wolfram syndrome?"
result = model.query_longcite(context, query, tokenizer=tokenizer, max_input_length=128000, max_new_tokens=512)

In [ ]:
answer = result['answer']
cleaned_answer = answer.replace("<statement>", "").replace("</statement>", "")
print(cleaned_answer)

Based on the context, the key clinical symptoms associated with Wolfram syndrome are:<cite></cite>

- Diabetes mellitus - Insulin-dependent diabetes<cite>[2-2][5-5][8-8]</cite>

- Diabetes insipidus - Inability to regulate water balance<cite>[2-2][5-5][8-8]</cite>

- Optic nerve atrophy - Progressive vision loss<cite>[2-2][5-5][8-8]</cite>

- Sensorineural hearing loss - Progressive deafness<cite>[2-2][5-5][8-8]</cite>

- Neurodegeneration - Progressive neurologic dysfunction<cite>[8-8]</cite>

- Olfactory dysfunction - Impaired sense of smell, although this is less well described<cite>[9-9]</cite>

- Other systemic manifestations - Urinary tract anomalies, endocrine dysfunctions, etc.<cite>[12-12]</cite>

So in summary, Wolfram syndrome is characterized by diabetes, vision and hearing loss, neurodegeneration, and other systemic manifestations. Olfactory dysfunction has been reported but is less well described.<cite></cite>


## Q5

In [ ]:
question = "What are some low-cost ways to screen individuals for prediabetes?"
RAG_PROMPT = process_question(question, db, top_k=5)

In [ ]:
print(RAG_PROMPT)

### Context: 
examination than those with non‐fulminant type 1 diabetes mellitus.
excellent opportunity to screen large numbers of individuals with a diagnosis of diabetes for mutations in disease-associated genes.
Diabetes, with its associated dyslipidemia and hypertension, is still not very well controlled. Screening for diabetes complications has improved over the years. Patients need to have more proper care, and physicians need to follow diabetes guidelines, and to have a larger number of patients who have appropriate treatment of diabetes, hypertension and lipids.
Only 1 patient with type 1 diabetes and 595 patients with type 2 diabetes were included in Wave 6. Average age was around 60 years, with a mean body mass index of 30. The mean fasting serum glucose was 159.42 mg/dL, and the mean glycosylated hemoglobin (HbA1c) level was 7.98 with around 30% achieving an HbA1c target of < 7%. More patients were on oral anti-diabetic medications. Screening of diabetic complications has im

###JSL_MedS_Rag

In [ ]:
response = jsl_meds_rag_llm.generate(RAG_PROMPT)
response

"1. Fasting Plasma Glucose Test: This test measures blood sugar levels after an overnight fast. It's a simple and cost-effective way to screen for prediabetes.\n\n2. Hemoglobin A1c Test: This test measures the average blood sugar levels over the past 2-3 months. It's a reliable indicator of long-term blood sugar control and can be used to screen for prediabetes.\n\n3. Oral Glucose Tolerance Test: This test measures blood sugar levels before and after drinking a sugary drink. It's more expensive and time-consuming than the other tests, but it can provide more detailed information about how the body processes glucose.\n\n4. Urine Test: This test checks for the presence of glucose in the urine, which can be an indicator of prediabetes. It's a simple and inexpensive test that can be done at home.\n\n5. Family History: If someone in your family has diabetes, you may be at a higher risk of developing the condition. Talk to your doctor about your family history and consider getting screened f

### JSL_MedS

In [ ]:
response = jsl_meds_llm.generate(RAG_PROMPT)
response

"1. Fasting Plasma Glucose (FPG) Test: This is a simple and cost-effective test that measures blood sugar levels after an overnight fast. It can be performed in a clinic or at home using a glucometer. A fasting blood sugar level of 100-125 mg/dL indicates prediabetes.\n2. Oral Glucose Tolerance Test (OGTT): This test involves measuring blood sugar levels before and two hours after consuming a glucose-rich drink. A 2-hour blood sugar level of 140-199 mg/dL indicates prediabetes. While more expensive and time-consuming than FPG, it provides a more comprehensive assessment of glucose metabolism.\n3. Hemoglobin A1c (HbA1c) Test: This test measures the average blood sugar levels over the past 2-3 months. An HbA1c level of 5.7%-6.4% indicates prediabetes. It is a convenient and cost-effective screening method that can be performed in a clinic or at home using a home HbA1c test.\n4. Urine Albumin-to-Creatinine Ratio (UACR): This test measures the amount of albumin (a type of protein) in the u

### JSL_MedM

In [ ]:
response = jsl_medm_llm.generate(RAG_PROMPT)
response

'The article does not specifically mention low-cost ways to screen individuals for prediabetes. However, it does mention that screening for diabetes complications has improved over the years, and that a large percentage of patients are diagnosed with hypertension and other complications. This suggests that there may be existing screening methods that are effective and cost-effective, but the article does not provide further details on these methods.'

### JSL-MedM_Rag

In [ ]:
response = jsl_medm_rag_llm.generate(RAG_PROMPT)
response

Based on the article, some low-cost ways to screen individuals for prediabetes include:

- Screening for diabetes complications has improved over the years. More patients are diagnosed with hypertension and dyslipidemia. This suggests routine blood pressure and lipid checks are an effective low-cost way to screen for prediabetes.

- The article notes that patients with diabetes need proper care and physicians need to follow diabetes guidelines. So promoting proper diabetes care and guideline-based management can help identify prediabetes.

- The article mentions that Wave 6 included 595 patients with type 2 diabetes. This suggests large-scale screening programs can identify prediabetes in high-risk populations.

- The article notes that around 30% of patients with diabetes achieve an HbA1c target of < 7%. This suggests screening for poor glycemic control can identify prediabetes.

- The article calls for elucidation of knowledge gaps in need of elucidation and exploring new ways to scr

### GPT-4

In [ ]:
# Generate the response using the model
response = generate_response(RAG_PROMPT)

In [ ]:
response_content = response.choices[0].message.content

print(response_content)

I don't know.


### LongCite

In [ ]:
context = RAG_PROMPT
query = "What are some low-cost ways to screen individuals for prediabetes?"
result = model.query_longcite(context, query, tokenizer=tokenizer, max_input_length=128000, max_new_tokens=512)

In [ ]:
answer = result['answer']
cleaned_answer = answer.replace("<statement>", "").replace("</statement>", "")
print(cleaned_answer)

Based on the context, some low-cost ways to screen individuals for prediabetes mentioned include:<cite></cite>

- Using HbA1c levels - The article notes that around 30% of patients achieved an HbA1c target of < 7%, which suggests HbA1c can be used to identify individuals with prediabetes.<cite>[7-7]</cite>

- Screening for hypertension and dyslipidemia - The article states that a large percentage of diabetics are diagnosed with hypertension and dyslipidemia, which are associated with prediabetes. Screening for these conditions can help identify prediabetes.<cite>[10-10]</cite>

- Using oral glucose tolerance tests (OGTTs) - While more expensive than HbA1c, OGTTs are a standard diagnostic tool for prediabetes. The article discusses the use of OGTTs in intervention trials.<cite></cite>

- Using fasting plasma glucose (FPG) - The article mentions FPG as a diagnostic criterion for prediabetes, although it is less accurate than HbA1c or OGTT.<cite></cite>

- Using lifestyle questionnaires -

## Q6

In [ ]:
question = "What are the most common malignancies associated with pituitary spread?"
RAG_PROMPT = process_question(question, db, top_k=5)

In [ ]:
print(RAG_PROMPT)

### Context: 
PM is a rare complication of advanced malignancy. Although several malignancies may spread to the pituitary, the most common are breast cancer in women and lung cancer in men. Unlike pituitary adenomas, which predominantly involve the anterior pituitary, PM has a predilection for the posterior lobe and infundibulum due to direct access via systemic circulation. The clinical presentation of PM depends on the size of the metastatic deposit and other structures involved in the vicinity of the
Pituitary metastases are rare, often deriving from lung or breast cancer owing to the upper vena cava proximity. Pituitary metastases can manifest with signs and symptoms of pituitary tumors, consequent to mass effect (headache, visual impairment) and/or hormonal alterations (hyperprolactinemia, hypopituitarism, and diabetes insipidus). Immune checkpoint inhibitors burst immunity against tumors, significantly increasing patients’ survival, but their autoimmune side effects frequently in

### JSL_MedS_Rag

In [ ]:
response = jsl_meds_rag_llm.generate(RAG_PROMPT)
response

'The most common malignancies associated with pituitary spread are breast cancer in women and lung cancer in men. These cancers are more likely to spread to the pituitary gland due to their proximity to the upper vena cava, which allows cancer cells to easily access the pituitary via systemic circulation.'

### JSL_MedS

In [ ]:
response = jsl_meds_llm.generate(RAG_PROMPT)
response

'The most common malignancies associated with pituitary spread are breast cancer in women and lung cancer in men. These cancers are more likely to spread to the pituitary gland due to their proximity to the upper vena cava. Other less common sources of pituitary metastases include gastrointestinal tumors, renal cell carcinoma, and less frequently, prostate and thyroid cancers. The spread of cancer to the pituitary gland can lead to various clinical manifestations, including hormonal imbalances and mass effects, which may result in symptoms such as headaches, visual disturbances, and other endocrine dysfunctions. Treatment for pituitary metastases often involves a combination of surgery, radiotherapy, and medical therapy, and the management of these patients can be complex due to the potential for autoimmune side effects from immune checkpoint inhibitors and other treatments.'

### JSL_MedM

In [ ]:
response = jsl_medm_llm.generate(RAG_PROMPT)
response

'The most common malignancies associated with pituitary spread are breast cancer in women and lung cancer in men.'

### JSL-MedM_Rag

In [ ]:
response = jsl_medm_rag_llm.generate(RAG_PROMPT)
response

Based on the context, the most common malignancies associated with pituitary spread are:

- Breast cancer in women

- Lung cancer in men

The context states that pituitary metastases are rare, and breast cancer is the most common malignancy to spread to the pituitary in women, while lung cancer is the most common in men. The text also notes that unlike pituitary adenomas, which are more common in the anterior pituitary, pituitary metastases tend to involve the posterior lobe and infundibulum due to direct access via systemic circulation.



### GPT-4

In [ ]:
# Generate the response using the model
response = generate_response(RAG_PROMPT)

In [ ]:
response_content = response.choices[0].message.content

print(response_content)

The most common malignancies associated with pituitary spread are breast cancer in women and lung cancer in men.


### LongCite

In [ ]:
context = RAG_PROMPT
query = "What are the most common malignancies associated with pituitary spread?"
result = model.query_longcite(context, query, tokenizer=tokenizer, max_input_length=128000, max_new_tokens=512)

In [ ]:
answer = result['answer']
cleaned_answer = answer.replace("<statement>", "").replace("</statement>", "")
print(cleaned_answer)

Based on the context, the most common malignancies associated with pituitary spread are:<cite></cite>

- Breast cancer in women<cite>[1-1]</cite>

- Lung cancer in men<cite>[1-1]</cite>

Pituitary metastases are rare, but when they occur, they are most commonly derived from breast or lung cancer due to the proximity of the upper vena cava. Other malignancies like gastrointestinal tumors and renal cell carcinoma can also rarely spread to the pituitary.<cite>[1-1][6-6]</cite>


## Q7

In [ ]:
question = "What is the recommended total daily dose of hydrocortisone for adults with adrenal insufficiency?"
RAG_PROMPT = process_question(question, db, top_k=5)

In [ ]:
print(RAG_PROMPT)

### Context: 
Plasma ACTH did not increase with insulin loading. A low plasma vasopressin (AVP) level and no response of AVP to a 5% saline administration were observed. We diagnosed central adrenal insufficiency with central diabetes insipidus. Six months after starting administration of hydrocortisone and 1-deamino-8D-arginine vasopressin, his psychological symptoms had improved, and 1.5 years after starting treatment, he was able to walk. In conclusion, it is not particularly rare for adrenal
signs. Adrenocorticotropic hormone deficiency replacement is best performed with the immediate-release oral glucocorticoid hydrocortisone (HC) in 2–3 divided doses. However, novel once-daily modified-release HC targets a more physiological exposure of glucocorticoids. GHD is treated currently with daily subcutaneous GH, but current research is focusing on the development of once-weekly administration of recombinant GH. Hypogonadism is targeted with testosterone replacement in men and on
A 36-da

### JSL_MedS_Rag

In [ ]:
response = jsl_meds_rag_llm.generate(RAG_PROMPT)
response

"The recommended total daily dose of hydrocortisone for adults with adrenal insufficiency is typically 10-20 mg, divided into two or three doses per day. The exact dosage may vary based on the individual's condition, response to treatment, and the presence of any other health issues. It is crucial to follow a healthcare provider's guidance when determining the appropriate dosage."

### JSL_MedS

In [ ]:
response = jsl_meds_llm.generate(RAG_PROMPT)
response

"The recommended total daily dose of hydrocortisone for adults with adrenal insufficiency is typically 17-20 mg divided into two or three doses throughout the day. This dosage may vary depending on the individual's specific needs and response to treatment. It is important for healthcare providers to closely monitor and adjust the dosage as necessary to achieve optimal hormone replacement and symptom management."

### JSL_MedM

In [ ]:
response = jsl_medm_llm.generate(RAG_PROMPT)
response

'The recommended total daily dose of hydrocortisone for adults with adrenal insufficiency is 20-30 mg.'

### JSL-MedM_Rag

In [ ]:
response = jsl_medm_rag_llm.generate(RAG_PROMPT)
response

Based on the case study, the recommended total daily dose of hydrocortisone for adults with adrenal insufficiency is not specified.

The case study focuses on a 36-day-old male infant who was treated with hydrocortisone for adrenal insufficiency. However, it does not provide details on the dose used in adults with adrenal insufficiency.

The American Association of Clinical Endocrinologists (AACE) recommends a total daily dose of hydrocortisone in the range of 15-30 mg for adults with adrenal insufficiency. However, the dose may need to be adjusted based on clinical response and serum cortisol levels.

In summary, while the case study does not provide specific details on the recommended total daily dose of hydrocortisone for adults with adrenal insufficiency, the AACE guidelines suggest a total daily dose in the range of 15-30 mg. The dose should be individualized based on clinical response and cortisol levels.



### GPT-4

In [ ]:
# Generate the response using the model
response = generate_response(RAG_PROMPT)

In [ ]:
response_content = response.choices[0].message.content

print(response_content)

I don't know.


### LongCite

In [ ]:
context = RAG_PROMPT
query = "What is the recommended total daily dose of hydrocortisone for adults with adrenal insufficiency?"
result = model.query_longcite(context, query, tokenizer=tokenizer, max_input_length=128000, max_new_tokens=512)

In [ ]:
answer = result['answer']
cleaned_answer = answer.replace("<statement>", "").replace("</statement>", "")
print(cleaned_answer)

Based on the information provided, the recommended total daily dose of hydrocortisone for adults with adrenal insufficiency is not explicitly stated.<cite></cite>

However, it mentions that adrenocorticotropic hormone deficiency replacement is best performed with immediate-release oral glucocorticoid hydrocortisone (HC) in 2-3 divided doses.<cite>[5-5]</cite>

So the typical dosing regimen for hydrocortisone in divided doses 2-3 times daily is recommended for adrenal insufficiency replacement.<cite></cite>

The exact total daily dose would depend on the individual patient's needs and medical condition. The dose may need to be adjusted up or down based on response and symptoms.<cite></cite>

So in summary, while a total daily dose is not explicitly stated, the typical dosing regimen for hydrocortisone in divided doses 2-3 times daily is recommended for adrenal insufficiency replacement, with individualized dose adjustments as needed.<cite></cite>


# MedicalLLM
You can also try below MedicalLLM annotator to load and run JSL large language models (LLMs).

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

medical_llm = MedicalLLM.pretrained("jsl_meds_q8_v1", "en", "clinical/models")\
    .setInputCols("document")\
    .setOutputCol("completions")\
    .setBatchSize(1)\
    .setNPredict(100)\
    .setUseChatTemplate(True)\
    .setTemperature(0.1)


pipeline = Pipeline(
    stages = [
        document_assembler,
        medical_llm
])


## Q1

In [ ]:

prompt = """
Based on the following text, What are the recommended measures to reduce the risk of GDM in pregnant women?

### Text:
of metabolic disease. One of the strongest risk factors for GDM is the occurrence of GDM in a prior pregnancy. Preliminary data from epidemiologic and bariatric surgery studies suggest that reducing body weight before pregnancy can prevent the development of GDM, but no adequately powered trial has tested the effects of a maternal lifestyle intervention before pregnancy to reduce body weight and prevent GDM recurrence.
outcomes and prevent GDM and associated health risks.
Implementation of dietary and lifestyle interventions prior to and early in pregnancy in high risk women has been shown to reduce the risk of gestational diabetes mellitus (GDM) development later in pregnancy. Although numerous risk factors for GDM have been identified, the ability to accurately identify women before or early in pregnancy who could benefit most from these interventions remains limited. As nulliparous women are an under-screened population with risk profiles that differ from
prenatal visit (P < 0.01) were identified as risk factors for GDM. In women with birth weight <3000 g, GDM rate was significantly higher.
One out of every five pregnant women in Beijing either had GDM or DIP and this constitutes a huge health burden for health services. Prepregnancy BMI and weight gain before 24th week are important modifiable risk factors for GDM. Ensuring birth weight above 3000 g may help reduce risk for future GDM among female offsprings.

"""

data = spark.createDataFrame([[prompt]]).toDF("text")

results = pipeline.fit(data).transform(data)

results.select("completions").show(truncate=False)

jsl_meds_q8_v1 download started this may take some time.
[OK!]
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|completions                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
+------------------------------------

In [ ]:
results.select("completions.result").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                                                                                                                                                                                                |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
result_text = results.select("completions.result").collect()[0]["result"]

result_text

[' The recommended measures to reduce the risk of GDM in pregnant women include:\n\n1. Reducing body weight before pregnancy: Preliminary data suggest that reducing body weight before pregnancy can prevent the development of GDM.\n\n2. Dietary and lifestyle interventions: Implementing dietary and lifestyle interventions prior to and early in pregnancy in high-risk women has been shown to reduce the risk']

## Q2


In [ ]:

prompt = """
Based on the following text, What are the main symptoms of diabetes?

## Text:
replacement therapy, the symptoms of diabetes insipidus appeared.
Almost all (99%) of the students had knowledge about diabetes mellitus. The majority (83.1%) reported that diabetes mellitus is not completely a genetic/hereditary disease. Only a minority of respondents reported that they should worry about diabetes before 45 years of age. Common symptoms of diabetes reported by the respondents included constant hunger, blurred vision, fatigue, and frequent urination.
Patient: Male, 2-year-old
 Final Diagnosis: Adipsic diabetes insipidus
 Symptoms: Dehydration • polyuria
 Medication: Desmopressin
 Clinical Procedure: Fluid replacement
 Specialty: Pediatrics and Neonatology • Endocrine and Metabolic
 Rare coexistence of disease or pathology
symptoms. She did report feeling hungry. She had no family history of diabetes, normal body mass index of 21.7, and reported taking her thyroid replacement medication every day. The differential diagnosis for her thirst included dehydration, psychogenic polydipsia, diabetes mellitus, diabetes insipidus, and anxiety. The patient had normal vital signs and was well appearing; labs were ordered for her on her way home from clinic with no medications. Labs revealed a random blood glucose level of
Primary care clinicians will see a higher incidence of type 2 diabetes in adult patients, and the diagnosis and management of an initial presentation of type 1 diabetes can pose challenges to clinicians who see it less frequently. Symptoms of hyperglycemia and risk of ketoacidosis may be missed. Further, endocrine autoimmune disease can run together in patients and families.

"""

data = spark.createDataFrame([[prompt]]).toDF("text")

results = pipeline.fit(data).transform(data)

results.select("completions").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|completions                                                                                                                                                              |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{document, 0, 125,  The main symptoms of diabetes mentioned in the text include constant hunger, blurred vision, fatigue, and frequent urination., {sentence -> 0}, []}]|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+



In [ ]:
result_text = results.select("completions.result").collect()[0]["result"]

result_text

[' The main symptoms of diabetes mentioned in the text include constant hunger, blurred vision, fatigue, and frequent urination.']

## Q3

In [ ]:

prompt = """
Based on the following text, What dietary changes can help manage diabetes??

## Text:
incidence of type 2 diabetes with higher intake of red meat (for an increment of 100 g/day, 1.17 (1.08 to 1.26)), processed meat (for an increment of 50 g/day, 1.37 (1.22 to 1.54)), bacon (per two slices/day, 2.07 (1.40 to 3.05)), and sugar sweetened beverages (for an increase of one serving/day, 1.26 (1.11 to 1.43)).
of 150 newly diagnosed cases attending a diabetes clinic, and 150 controls, who were community residents and free of diabetes. The case group had a significantly higher consumption of noodles, fish, beans, fermented food and pickles, dried food, topping seasonings, and non-dairy milk products than the control group, whereas they had a lower vegetable intake (more than three servings/day) and fruit intake (more than three servings/day) than the control group. Furthermore, the case group
income, there is a need for a certain affordable dietary pattern to ameliorate the rise in diabetes incidence, as well as improve the glycemic control. We did a review based on the available literature regarding Mediterranean diet and people with or at risk of developing type 2 diabetes mellitus, its effects on glycemic control, lipid profile and metabolic outcome.
management approaches include lifestyle changes, such as improved diet and increased physical activity, together with pharmacological interventions, including metformin, insulin, and the recently approved glucagon-like peptide-1 analog liraglutide. What is more important for everyone to realize though, from patients, families and physicians to schools, health services and policy-makers alike, is that T2DM is a largely preventable disease that will be addressed effectively only if its major
relevant pathway in human patients suffering from diabetes.

"""

data = spark.createDataFrame([[prompt]]).toDF("text")

results = pipeline.fit(data).transform(data)

results.select("completions").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|completions                                                                                                                                                                                                                                                                                                                                                                                                |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
result_text = results.select("completions.result").collect()[0]["result"]

result_text

[' The text suggests that dietary changes can help manage diabetes. Specifically, it mentions the following:\n\n1. Reducing intake of red meat, processed meat, bacon, and sugar-sweetened beverages.\n2. Increasing consumption of noodles, fish, beans, fermented food, pickles, dried food, topping seasonings, and non-dairy milk products.\n3. Lowering veget']